In [1]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP#降维工具
from hdbscan import HDBSCAN#聚类
from bertopic.vectorizers import ClassTfidfTransformer#话题包
from tqdm import tqdm
from transformers import BertTokenizer,BertModel
from sklearn.feature_extraction.text import CountVectorizer
with open('text1_data_1_3.txt', 'r', encoding='utf-8') as file:#导入切词后的文档
  docs = file.readlines()
stopwords=[line.strip() for line in open('hit_stopwords.txt',encoding='UTF-8').readlines()]

In [7]:
embedding_model = SentenceTransformer(
  'paraphrase-MiniLM-L12-v2',
)#实例化模型
umap_model = UMAP(n_neighbors=15, n_components=5,min_dist=0.0,metric='cosine',random_state=10)#实例化UMAP
hdbscan_model = HDBSCAN(min_cluster_size=40)  #实例化聚类算法
embeddings = np.load('embedding_data_1_3.npy')#导入之间做好的词向量


vectorizer_model = CountVectorizer(stop_words=stopwords)#停用词

 
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  hdbscan_model=hdbscan_model,
  umap_model=umap_model,
)

topics,probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,237,-1_参数_ffc_步骤_充电,"[参数, ffc, 步骤, 充电, 电压, 连接, mcu, 模块, can, 装置]",[申请 实施 例中 提供 一种 动力电池 充电 方法 电池 管理系统 能够 有效 保障 电池...
1,0,13629,0_连接_安装_装置_车辆,"[连接, 安装, 装置, 车辆, 组件, 系统, 充电, 控制, 固定, 实用新型]",[实用新型 公开 一种 防护 电芯 模组 电池 涉及 动力电池 技术 领域 实用新型 防护 ...
2,1,470,1_外观设计_产品_要点_设计,"[外观设计, 产品, 要点, 设计, 最能, 照片, 用途, 图片, 名称, 表明]",[外观设计 产品 名称 动力电池 外观设计 产品 用途 用于 新能源 动力电池 外观设计 产...
3,2,359,2_dc_电路_模块_变换器,"[dc, 电路, 模块, 变换器, 系统, 充电, 连接, 控制, 双向, dcdc]",[实用新型 提供 一种 用于 车载 燃料电池 运行 控制 装置 属于 燃料电池 发动机 技术...
4,3,331,3_装置_连接_用于_车辆,"[装置, 连接, 用于, 车辆, 控制, 充电, 电机, 驱动, 回路, 壳体]",[公开 一种 用于 牵引 电池 模块 自动 拆卸 设备 设备 包括 连接 抓取 装置 抓取 ...
5,4,283,4_bms_模块_bdu_系统,"[bms, 模块, bdu, 系统, 继电器, 控制器, 高压, 连接, 充电, 控制]",[实用新型 提供 一种 锂离子 动力电池 系统 涉及 电池 技术 领域 包括 多个 电池 单...
6,5,252,5_步骤_模型_数据_系统,"[步骤, 模型, 数据, 系统, 以下, 状态, 得到, 检测, 基于, 车辆]",[本发明 公开 基于 集成 深度 学习 动力电池 SOH 估算 方法 涉及 充电 安全 技术...
7,6,248,6_重量_组分_制备_材料,"[重量, 组分, 制备, 材料, 导热, 聚氨酯, 具有, 原料, 阻燃, 改性]",[本发明 提供 一种 双组分 聚氨酯 导热 结构胶 包括 体积 0.8 1.2 组分 组分 ...
8,7,104,7_fpc_采集_ctp_ccs,"[fpc, 采集, ctp, ccs, 连接, 组件, 结构, pcb, 电芯, 模组]",[申请 公开 一种 FPC 采集 装置 动力电池 车辆 所述 FPC 采集 装置 包括 FP...


In [9]:
topic_docs.to_csv('./聚类结果.csv')#传出聚类结果，使用excel打开CSV会报错

In [15]:
topic_model.merge_topics(docs, [
  [3, 0]
])#手动合并话题
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,237,-1_参数_ffc_步骤_充电,"[参数, ffc, 步骤, 充电, 电压, 连接, 模块, mcu, 装置, 数据]",[申请 实施 例中 提供 一种 动力电池 充电 方法 电池 管理系统 能够 有效 保障 电池...
1,0,14243,0_连接_安装_装置_车辆,"[连接, 安装, 装置, 车辆, 系统, 充电, 控制, 组件, 固定, 实用新型]",[申请 实施 提供 一种 动力 电池组 短路 保护 电路 方法 装置 电路 包括 电流 检测...
2,1,470,1_外观设计_产品_要点_设计,"[外观设计, 产品, 要点, 设计, 最能, 照片, 用途, 图片, 名称, 表明]",[外观设计 产品 名称 动力电池 外观设计 产品 用途 用于 新能源 动力电池 外观设计 产...
3,2,359,2_dc_电路_模块_系统,"[dc, 电路, 模块, 系统, 变换器, 充电, 连接, 控制, 双向, 燃料电池]",[实用新型 提供 一种 用于 车载 燃料电池 运行 控制 装置 属于 燃料电池 发动机 技术...
4,3,252,3_步骤_模型_数据_系统,"[步骤, 模型, 数据, 系统, 以下, 状态, 得到, 检测, 车辆, 基于]",[本发明 涉及 一种 基于 实车 数据 动力电池 评估 方法 方法 包括 以下 步骤 步骤 ...
5,4,248,4_重量_组分_制备_材料,"[重量, 组分, 制备, 材料, 导热, 聚氨酯, 具有, 原料, 阻燃, 改性]",[本发明 提供 一种 双组分 聚氨酯 导热 结构胶 包括 体积 0.8 1.2 组分 组分 ...
6,5,104,5_fpc_采集_ctp_连接,"[fpc, 采集, ctp, 连接, ccs, 组件, 结构, 电芯, pcb, 模组]",[申请 公开 一种 FPC 采集 装置 动力电池 车辆 所述 FPC 采集 装置 包括 FP...


In [16]:
topic_model.visualize_topics()#可视化